# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [13]:
# Cell-to-cell script
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [14]:
# Importing libraries
import pandas as pd
import numpy as np
#Loading data
data = pd.read_csv('1.6mill_tweet_training.csv') 
data.columns = ['target','id','date','flag','user','text']

data.head()
data.shape

,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


(1599999, 6)

In [15]:
df=data.sample(20000)
df.shape

(20000, 6)

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [16]:
import regex as re

In [17]:
def clean_up(text):
    text= re.sub(r"http\S+", "", text) #delete the web
    text=re.sub('\d+',' ',text) # space x numbers
    text=re.sub('\W+',' ',text) # space for   special characters
    text=text.lower()
    return text

In [18]:
df['text_processed']=df['text'].apply(clean_up)

In [19]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def tokenize(string):
    tokens= word_tokenize(string)
    return tokens

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Elena\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
df['text_processed']=df['text_processed'].apply(tokenize)

In [21]:
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Elena\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [22]:
# nltk.download('averaged_perceptron_tagger')

# def get_wordnet_pos(word):
#     tag = nltk.pos_tag([word])[0][1][0].upper() # gets first letter of POS categorization
#     tag_dict = {"J": wordnet.ADJ, 
#                 "N": wordnet.NOUN,
#                 "V": wordnet.VERB,
#                 "R": wordnet.ADV}
#     return tag_dict.get(tag, wordnet.NOUN) # get returns second argument if first key does not exist

In [23]:
# df['text_processed']=df['text_processed'].apply(get_wordnet_pos)

In [24]:
def stem_and_lemmatize(tokens):
    ps = PorterStemmer()
    stemmed = [ps.stem(w) for w in tokens]
    lemmatizer = WordNetLemmatizer() 
    lemmatized = [lemmatizer.lemmatize(word) for word in stemmed]
    return lemmatized

In [25]:
df['text_processed']=df['text_processed'].apply(stem_and_lemmatize)

In [26]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Elena\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
def remove_stopwords(l):
    
    filtered_sentence = []
    
    for w in l:
        if w not in stopwords.words('english'):
            filtered_sentence.append(w)
    
    return filtered_sentence

In [28]:
df['text_processed']=df['text_processed'].apply(remove_stopwords)

In [29]:
df.head()

,target,id,date,flag,user,text,text_processed
499988,0,2186707353,Mon Jun 15 19:13:17 PDT 2009,NO_QUERY,RENNIROO,eating haagen-dazs! long day tomorow i NEE...,"[eat, haagen, daz, long, day, tomorow, need, b..."
326163,0,2008025147,Tue Jun 02 13:30:32 PDT 2009,NO_QUERY,lorettabreman,"back to real world, hope everyone is good?? J...","[back, real, world, hope, everyon, good, june,..."
1017828,4,1881915626,Fri May 22 05:50:52 PDT 2009,NO_QUERY,BabyVanessa093,@HazalSelena haha I dunno but it's funny dam...,"[hazalselena, haha, dunno, funni, damn, faast,..."
1293117,4,2003157764,Tue Jun 02 05:59:57 PDT 2009,NO_QUERY,marcusblade3,@PMSHarlequin Your going to E3 Right.if you ar...,"[pmsharlequin, go, e, right, tweet]"
1089803,4,1969738331,Sat May 30 01:02:49 PDT 2009,NO_QUERY,zibbet,I'm off for the day...Tweet you all later! Ha...,"[day, tweet, later, relax, weekend]"


In [30]:
df['text_processed'].iloc[0]

['eat',
 'haagen',
 'daz',
 'long',
 'day',
 'tomorow',
 'need',
 'buy',
 'new',
 'jb',
 'album',
 'tomorow',
 'el',
 'wont',
 'abl',
 'get',
 'bed']

In [31]:
df['target'].value_counts()

0    10076
4     9924
Name: target, dtype: int64

In [32]:
df['target']= df['target'].replace (4,1)

In [33]:
df['target'].value_counts()

0    10076
1     9924
Name: target, dtype: int64

### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [37]:
# # class FreqDist(Counter):
    
#     """
#     A frequency distribution for the outcomes of an experiment.  A
#     frequency distribution records the number of times each outcome of
#     an experiment has occurred.  For example, a frequency distribution
#     could be used to record the frequency of each word type in a
#     document.  Formally, a frequency distribution can be defined as a
#     function mapping from each sample to the number of times that
#     sample occurred as an outcome.

#     Frequency distributions are generally constructed by running a
#     number of experiments, and incrementing the count for a sample
#     every time it is an outcome of an experiment.  For example, the
#     following code will produce a frequency distribution that encodes
#     how often each word occurs in a text:

#         >>> from nltk.tokenize import word_tokenize
#         >>> from nltk.probability import FreqDist
#         >>> sent = 'This is an example sentence'
#         >>> fdist = FreqDist()
#         >>> for word in word_tokenize(sent):
#         ...    fdist[word.lower()] += 1

In [38]:
from nltk.probability import FreqDist
fdist = FreqDist()

for tweet in df['text_processed']:
    for word in tweet:
        fdist[word] += 1

top_words = list(fdist.keys())[:5000]
top_words

['eat',
 'haagen',
 'daz',
 'long',
 'day',
 'tomorow',
 'need',
 'buy',
 'new',
 'jb',
 'album',
 'el',
 'wont',
 'abl',
 'get',
 'bed',
 'back',
 'real',
 'world',
 'hope',
 'everyon',
 'good',
 'june',
 'never',
 'know',
 'thi',
 'weather',
 'hazalselena',
 'haha',
 'dunno',
 'funni',
 'damn',
 'faast',
 'take',
 'much',
 'longer',
 'o_o',
 'pmsharlequin',
 'go',
 'e',
 'right',
 'tweet',
 'later',
 'relax',
 'weekend',
 'softbal',
 'game',
 'awesom',
 'start',
 'morn',
 'hey',
 'mileycyru',
 'support',
 'www',
 'twitter',
 'com',
 'x',
 'neglect',
 'sorri',
 'appl',
 'store',
 'love',
 'job',
 'suzydafloozi',
 'dont',
 'point',
 'yet',
 'wa',
 'talkedabout',
 'though',
 'first',
 'ticket',
 'whole',
 'drive',
 'career',
 'date',
 'bad',
 'mcooper',
 'true',
 'fine',
 'winter',
 'sick',
 'fml',
 'mahhriahh',
 'homepag',
 'today',
 'gon',
 'na',
 'suuuuuuck',
 'work',
 'tire',
 'almost',
 'miss',
 'train',
 'still',
 'pm',
 'thought',
 'goin',
 'auckland',
 'actual',
 'tomorrow',
 'o

In [39]:
len(top_words)

5000

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [40]:
#Next, we're going to build a quick function that will find these top 5,000 words in our positive and negative documents, marking their presence as either positive or negative:

In [41]:
def find_features(document):
    words = set(document) # we convert to a set to iterate to unique elements (unique words).
    #The doc is our text_processed column
    features = {} # this is an empty dictionary
    for w in top_words:#
        features[w] = (w in words) # in this features dictionary the key is the word.
        #this creates a boolean ( T/F) so if the word is in this top_word list is T if not is F

    return features

In [42]:
feature_sets = [(find_features(tweet), target) for (tweet, target) in list(zip(df['text_processed'], df['target']))]

In [43]:
feature_sets[10]

({'eat': False,
  'haagen': False,
  'daz': False,
  'long': False,
  'day': False,
  'tomorow': False,
  'need': False,
  'buy': False,
  'new': False,
  'jb': False,
  'album': False,
  'el': False,
  'wont': False,
  'abl': False,
  'get': True,
  'bed': False,
  'back': False,
  'real': False,
  'world': False,
  'hope': False,
  'everyon': False,
  'good': False,
  'june': False,
  'never': False,
  'know': False,
  'thi': False,
  'weather': False,
  'hazalselena': False,
  'haha': False,
  'dunno': False,
  'funni': False,
  'damn': False,
  'faast': False,
  'take': False,
  'much': False,
  'longer': False,
  'o_o': False,
  'pmsharlequin': False,
  'go': False,
  'e': False,
  'right': False,
  'tweet': False,
  'later': False,
  'relax': False,
  'weekend': False,
  'softbal': False,
  'game': False,
  'awesom': False,
  'start': False,
  'morn': False,
  'hey': False,
  'mileycyru': False,
  'support': False,
  'www': False,
  'twitter': False,
  'com': False,
  'x': False,

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [44]:
# set that we'll train our classifier with
training_set = feature_sets[:10000]

# set that we'll test against.
testing_set = feature_sets[10000:]

In [45]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [51]:
classifier.show_most_informative_features(15)

Most Informative Features
                  welcom = True                1 : 0      =     18.6 : 1.0
                     sad = True                0 : 1      =     16.8 : 1.0
                   upset = True                0 : 1      =     12.7 : 1.0
                     due = True                0 : 1      =     11.4 : 1.0
                     fml = True                0 : 1      =     11.4 : 1.0
                   proud = True                1 : 0      =     11.2 : 1.0
                    glad = True                1 : 0      =     11.0 : 1.0
                  broken = True                0 : 1      =     10.8 : 1.0
                    wors = True                0 : 1      =     10.8 : 1.0
                 congrat = True                1 : 0      =     10.4 : 1.0
                   stuck = True                0 : 1      =     10.4 : 1.0
                  throat = True                0 : 1      =     10.1 : 1.0
                 headach = True                0 : 1      =      9.9 : 1.0

In [ ]:
# What this tells you is the ratio of occurences in negative to positive, or visa versa, for every word. 
#So here, we can see that the term "welcom" appears 18.6 more times as often in positive reviews as it does in negative reviews. 
#The term "sad" appears 16.8 more times as often in positive reviews as it does in negative reviews. 

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [52]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

Classifier accuracy percent: 72.39999999999999


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [48]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [49]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [50]:
# your code here